In [39]:
from modules import abrindo_dados, adicionando_dados, caixa_de_ferramentas_vetores, pos_analise_de_dados
import os
import geopandas as gpd
import pandas as pd

In [40]:
#Abrindo arquivos
vault_files = "/home/luisthethormes/01_sol/geo-dev/Sol-luis.github.io/data/"

limite_produtor = abrindo_dados.process_files(vault_files, "pol_props*", "geojson", zipped = False)
limite_produtor = limite_produtor[['car','produtor', 'municipio', 'des_condic', 'area_hectares', 'geometry']]
desmates = abrindo_dados.process_files(vault_files, "prodes_deforestation_*", "geojson", zipped = False)
desmates = desmates[['year', 'image_date', 'geometry']]
es_uso_solo_vetor = abrindo_dados.process_files(vault_files, "es_uso_solo_vetor_reclassificado_pt*", "geojson", zipped = False)

Dimensões do(s) arquivo(s) ['pol_props_ES.geojson']: (74, 15)
Dimensões do(s) arquivo(s) ['prodes_deforestation_clip.geojson']: (60, 16)
Dimensões do(s) arquivo(s) ['es_uso_solo_vetor_reclassificado_pt.geojson']: (1761, 3)


In [41]:
### Calculando área em hectares ###
desmates = adicionando_dados.add_area(desmates)
es_uso_solo_vetor = adicionando_dados.add_area(es_uso_solo_vetor)

In [42]:
#Unindo espacialmente
limite_produtor_left_desmates = caixa_de_ferramentas_vetores.spatial_joining(limite_produtor, desmates, how='left', predicate='intersects')
limite_produtor_left_uso_solo = caixa_de_ferramentas_vetores.spatial_joining(limite_produtor, es_uso_solo_vetor, how='left', predicate='intersects')

crs are the same
crs are the same


In [43]:
limite_produtor_left_desmates[['area_hectares_right', 'year']] =  limite_produtor_left_desmates[['area_hectares_right', 'year']].fillna(0)
limite_produtor_left_desmates['year'] = limite_produtor_left_desmates['year'].astype(int)

In [44]:
#pivoting results by year and car while suming area_hectares
pivoted_deforestation = pd.pivot_table(limite_produtor_left_desmates,
                                        index=['car','produtor', 'municipio','area_hectares_left'],
                                          columns='year',
                                            values='area_hectares_right',
                                              aggfunc='sum',
                                                dropna = True,
                                                  fill_value=0)
pivoted_deforestation_r = pivoted_deforestation.reset_index()
pivoted_deforestation_r[[2009, 2012, 2015, 2017, 2020, 2022]] = 0.0

### Reordenando df
pivoted_deforestation_final = pivoted_deforestation_r[['car',
                                                        'produtor',
                                                          'municipio',
                                                            'area_hectares_left',
                                                             2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
                                                             2017, 2018, 2019, 2020, 2021, 2022, 2023]]

pivoted_deforestation_final = pivoted_deforestation_final.rename (columns={'area_hectares_left' : 'area da propriedade rural (ha)', 
                                                                            'car': 'código CAR'                                                                           
                                                                           })
pivoted_deforestation_final.to_excel(os.path.join(vault_files, 'Análise desmatamento série (2008-2023).xlsx'))

In [45]:
stats = pos_analise_de_dados.calculate_stats(pivoted_deforestation_final, stats=['mean', 'median', 'std', 'max'])
stats.to_excel(os.path.join(vault_files, 'Estatísticas desmatamento série (2008-2023).xlsx'))

In [46]:
limite_produtor_left_uso_solo = pd.pivot_table(limite_produtor_left_uso_solo,
                                        index=['car','produtor', 'municipio','area_hectares_left'],
                                          columns='classe_uso_solo_mapbiomas',
                                            values='area_hectares_right',
                                              aggfunc='sum',
                                                  fill_value=0)                                                
limite_produtor_left_uso_solo_r = limite_produtor_left_uso_solo.reset_index()
limite_produtor_left_uso_solo_r.to_excel(os.path.join(vault_files, 'Análise uso do solo série(2023).xlsx'))

In [47]:
stats = pos_analise_de_dados.calculate_stats(limite_produtor_left_uso_solo_r, stats=['mean', 'median', 'std', 'max'])
stats.to_excel(os.path.join(vault_files, 'Estatísticas uso do solo série (2023).xlsx'))